# Virtual Screening Results Analysis

This notebook presents the results of the virtual screening campaign comparing:
- **Vina**: Traditional docking score (lower = better binding)
- **AEV-PLIG**: Machine learning rescoring (higher = better binding)

---

## 1. Setup & Data Loading

In [ ]:
import pandas as pd
import numpy as np
import pyarrow.parquet as pq
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from IPython.display import display, Markdown

# Plot settings
plt.rcParams.update({
    'figure.figsize': (10, 6),
    'font.size': 12,
    'axes.titlesize': 14,
    'axes.labelsize': 12,
})
sns.set_style('whitegrid')

# Paths
PROJECT_ROOT = Path('..').resolve()
MANIFEST_PATH = PROJECT_ROOT / 'data/master/manifest.parquet'
RESULTS_DIR = PROJECT_ROOT / 'results'

print(f"Project root: {PROJECT_ROOT}")
print(f"Manifest: {MANIFEST_PATH}")
print(f"Results: {RESULTS_DIR}")

In [ ]:
# Load manifest
manifest = pq.read_table(MANIFEST_PATH).to_pandas()

# Filter to rescored compounds
df = manifest[manifest['rescoring_status'] == True].copy()
if len(df) == 0:
    print("Warning: No rescored compounds. Using docked compounds.")
    df = manifest[manifest['docking_status'] == True].copy()

# Add convenience columns
df['Vina'] = df['vina_score']
df['AEV-PLIG'] = df['aev_plig_best_score']

print(f"Loaded {len(df):,} compounds")

In [ ]:
# Load computed metrics
per_target = pd.read_csv(RESULTS_DIR / 'per_target_metrics.csv')
summary = pd.read_csv(RESULTS_DIR / 'summary.csv')

print(f"Loaded metrics for {len(per_target)} targets")

## 2. Executive Summary

In [ ]:
# Key statistics
n_targets = df['protein_id'].nunique()
n_compounds = len(df)
n_actives = df['is_active'].sum()
active_rate = 100 * n_actives / n_compounds

display(Markdown(f"""
### Dataset Overview

| Metric | Value |
|--------|-------|
| Targets | {n_targets} |
| Compounds | {n_compounds:,} |
| Actives | {n_actives:,} ({active_rate:.1f}%) |
| Inactives | {n_compounds - n_actives:,} |
"""))

In [ ]:
# Summary metrics table
display(Markdown("### Performance Comparison"))

display_cols = ['Metric', 'Vina', 'AEV-PLIG', 'p_value', 'Significant']
display_cols = [c for c in display_cols if c in summary.columns]
display(summary[display_cols].style.set_caption("Median [95% CI] across targets"))

## 3. Overall Performance

In [ ]:
# Load and display ROC curves
from IPython.display import Image

roc_path = RESULTS_DIR / 'plots' / 'average_ROC.png'
if roc_path.exists():
    display(Markdown("### Average ROC Curves"))
    display(Image(filename=str(roc_path), width=600))
else:
    print(f"ROC plot not found at {roc_path}")

In [ ]:
# Load and display PRC curves
prc_path = RESULTS_DIR / 'plots' / 'average_PRC.png'
if prc_path.exists():
    display(Markdown("### Average Precision-Recall Curves"))
    display(Image(filename=str(prc_path), width=600))
else:
    print(f"PRC plot not found at {prc_path}")

## 4. Early Enrichment Analysis

Early enrichment is the most important metric for virtual screening - it measures how well the method ranks actives at the top of the list.

In [ ]:
# Enrichment factor comparison
ef_metrics = ['EF1%', 'EF5%', 'EF10%']
ef_data = []

for metric in ef_metrics:
    for method in ['Vina', 'AEV-PLIG']:
        col = f'{method}_{metric}'
        if col in per_target.columns:
            values = per_target[col].dropna()
            ef_data.append({
                'Metric': metric,
                'Method': method,
                'Mean': values.mean(),
                'Std': values.std(),
            })

ef_df = pd.DataFrame(ef_data)

fig, ax = plt.subplots(figsize=(10, 6))
x = np.arange(len(ef_metrics))
width = 0.35

vina_means = ef_df[ef_df['Method'] == 'Vina']['Mean'].values
aevplig_means = ef_df[ef_df['Method'] == 'AEV-PLIG']['Mean'].values
vina_stds = ef_df[ef_df['Method'] == 'Vina']['Std'].values
aevplig_stds = ef_df[ef_df['Method'] == 'AEV-PLIG']['Std'].values

ax.bar(x - width/2, vina_means, width, yerr=vina_stds, label='Vina', color='#3498db', capsize=5)
ax.bar(x + width/2, aevplig_means, width, yerr=aevplig_stds, label='AEV-PLIG', color='#e74c3c', capsize=5)

ax.axhline(y=1, color='gray', linestyle='--', label='Random')
ax.set_ylabel('Enrichment Factor')
ax.set_title('Early Enrichment Comparison')
ax.set_xticks(x)
ax.set_xticklabels(ef_metrics)
ax.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Calculate "actives in top 1%" headline number
display(Markdown("### Key Finding: Actives in Top 1%"))

for method in ['Vina', 'AEV-PLIG']:
    col = f'{method}_EF1%'
    if col in per_target.columns:
        mean_ef = per_target[col].mean()
        # EF1% = (actives in top 1%) / (expected actives in top 1%)
        # So actives_found = EF * expected = EF * (n_actives * 0.01)
        print(f"{method}: {mean_ef:.1f}x more actives in top 1% than random")

## 5. Per-Target Analysis

In [ ]:
# Heatmap of performance by target
metrics_to_show = ['ROC-AUC', 'BEDROC', 'EF1%', 'NEF1%']

# Build comparison dataframe
comparison_data = []
for _, row in per_target.iterrows():
    target = row['Target']
    for metric in metrics_to_show:
        vina_col = f'Vina_{metric}'
        aevplig_col = f'AEV-PLIG_{metric}'
        if vina_col in row and aevplig_col in row:
            vina_val = row[vina_col]
            aevplig_val = row[aevplig_col]
            if pd.notna(vina_val) and pd.notna(aevplig_val):
                diff = aevplig_val - vina_val
                comparison_data.append({
                    'Target': target,
                    'Metric': metric,
                    'Difference': diff,  # Positive = AEV-PLIG better
                })

comparison_df = pd.DataFrame(comparison_data)

if len(comparison_df) > 0:
    pivot = comparison_df.pivot(index='Target', columns='Metric', values='Difference')
    
    fig, ax = plt.subplots(figsize=(10, max(6, len(pivot) * 0.4)))
    sns.heatmap(pivot, cmap='RdYlGn', center=0, annot=True, fmt='.2f', ax=ax)
    ax.set_title('AEV-PLIG vs Vina (positive = AEV-PLIG better)')
    plt.tight_layout()
    plt.show()

In [ ]:
# Best and worst targets for AEV-PLIG
display(Markdown("### Targets Where AEV-PLIG Performs Best"))

if 'AEV-PLIG_ROC-AUC' in per_target.columns and 'Vina_ROC-AUC' in per_target.columns:
    per_target['AUC_improvement'] = per_target['AEV-PLIG_ROC-AUC'] - per_target['Vina_ROC-AUC']
    
    top_5 = per_target.nlargest(5, 'AUC_improvement')[['Target', 'N_Compounds', 'N_Actives', 'Vina_ROC-AUC', 'AEV-PLIG_ROC-AUC', 'AUC_improvement']]
    display(top_5)
    
    display(Markdown("### Targets Where Vina Performs Better"))
    bottom_5 = per_target.nsmallest(5, 'AUC_improvement')[['Target', 'N_Compounds', 'N_Actives', 'Vina_ROC-AUC', 'AEV-PLIG_ROC-AUC', 'AUC_improvement']]
    display(bottom_5)

## 6. Score Distributions

In [ ]:
# Score distributions for actives vs inactives
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Vina scores
ax = axes[0]
actives = df[df['is_active'] == True]['Vina'].dropna()
inactives = df[df['is_active'] == False]['Vina'].dropna()

ax.hist(inactives, bins=50, alpha=0.7, label=f'Inactives (n={len(inactives):,})', color='gray')
ax.hist(actives, bins=50, alpha=0.7, label=f'Actives (n={len(actives):,})', color='green')
ax.set_xlabel('Vina Score (kcal/mol)')
ax.set_ylabel('Count')
ax.set_title('Vina Score Distribution (lower = better)')
ax.legend()

# AEV-PLIG scores
ax = axes[1]
actives = df[df['is_active'] == True]['AEV-PLIG'].dropna()
inactives = df[df['is_active'] == False]['AEV-PLIG'].dropna()

ax.hist(inactives, bins=50, alpha=0.7, label=f'Inactives (n={len(inactives):,})', color='gray')
ax.hist(actives, bins=50, alpha=0.7, label=f'Actives (n={len(actives):,})', color='green')
ax.set_xlabel('AEV-PLIG Score')
ax.set_ylabel('Count')
ax.set_title('AEV-PLIG Score Distribution (higher = better)')
ax.legend()

plt.tight_layout()
plt.show()

## 7. Recommendations

In [ ]:
# Generate recommendations based on results
display(Markdown("""
### Summary & Recommendations

Based on the analysis above:

1. **Overall Performance**: [To be filled based on results]

2. **Early Enrichment**: [To be filled based on EF1% results]

3. **Target-Specific Observations**: [To be filled based on per-target analysis]

4. **Recommended Workflow**:
   - Use Vina for initial docking
   - Apply AEV-PLIG rescoring to improve ranking
   - Select top N compounds for experimental validation

5. **Suggested Score Cutoffs**:
   - [To be determined based on ROC analysis]

### Next Steps

- [ ] Select top compounds for experimental testing
- [ ] Validate predictions with binding assays
- [ ] Iterate on model if needed
"""))